# About
This notebook contains a workflow to add contrast and correlation from the gray-level co-occurrence matrix (GLCM) around a point in a NAIP scene. Assumes all the points in the csv have the same crs.


It appends these features as columns to the given dataset and saves result in temp folder.

**NOTEBOOK PARAMETERS**

-`data_path` (str): filepath to the csv file containing the points for which we want to calculate GLCM texture features for NAIP bands over a window
- `itemid_col` (str): name of column containing itemid of the NAIP scene containing the point
- `crs_col` (str): name of columns with the crs of all points
- `lon_label` (str): column names for longitude
- `lat_label` (str): column names for latitude
- `window_r` (array of int): radius of the window(s) (in pixels) over which entropy is calculated. For each given radius r the GLCM be calculated over a box centered at the point with side length 2r+1.
- `distances` (array of int): pixel offset when computing GLCM
- `angles` (array of float): the angles used to compute the GLCM matrix(ces)
- `props` (array of str): The properties of the GLCM to compute. See scikit-image [`skimage.feature.graycoprops` documentation](https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycoprops)

**OUTPUT**

## References

- [GLCM TEXTURE: A TUTORIAL](https://prism.ucalgary.ca/bitstream/handle/1880/51900/texture%20tutorial%20v%203_0%20180206.pdf?sequence=11&isAllowed=y) by Mryka Hall-Beyer
- Practical guidelines for choosing GLCM textures to use in landscape classification tasks over a range of moderate spatial scales by Mryka Hall-Beyer https://doi.org/10.1080/01431161.2016.1278314
- scikit-image [GLCM example](https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_glcm.html)
- [A stackoverflow example for calculating GLCM features](https://stackoverflow.com/questions/50834170/image-texture-with-skimage)

In [ ]:
import os
import pandas as pd
import numpy as np

import geopandas as gpd
import rioxarray as rioxr
import rasterio

import sample_rasters as sr
from rasterio.crs import CRS
from rasterio.transform import rowcol

from shapely.geometry import box

import planetary_computer as pc

from skimage.feature import graycomatrix, graycoprops

In [ ]:
# Assuming repository's parent directory is the home directory
home = os.path.expanduser("~")
os.chdir(os.path.join(home,'iceplant-detection-santa-barbara'))

# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

# csv with the points for which to add GLCM features
data_path =  os.path.join(os.getcwd(),
                   'notebooks',
                   'A_data_sampling',
                   'output',
                   'spectral_windows_iceplant_pts.csv')

# -------------------------------------------
# name of column containing itemid of the NAIP scene containing the point if there is one
itemid_col = 'naip_id'
# name ofcolumns with the crs of all points
crs_col = 'pts_crs'

# -------------------------------------------
# column names for longitude and latitude
lon_label = 'x' # lon = x
lat_label = 'y' # lat = y

# -------------------------------------------
# GLCM parameters
window_r = [1,2] 
distances = [1]
angles = [0, np.pi/2]
props = ['contrast', 'correlation']

# ***************************************************
# ***************************************************

In [ ]:
# ===================================================
def clear_screen():
    if os.name in ('nt', 'dos'):
        _ = os.system('cls')
    elif os.name == 'posix':
        _ = os.system('clear')
        
# ===================================================
# folder for output points
out_dir = os.path.join(os.getcwd(),
                       'notebooks',
                       'A_data_sampling',
                       'output')
if os.path.exists(out_dir) == False:
    os.mkdir(out_dir)    

# ===================================================
# these are ordered as they appear in the for loop to calculate correlation and contrast
columns = ['r_contE', 'r_corrE', 'g_contE', 'g_corrE', 'b_contE', 'b_corrE', 'nir_contE', 'nir_corrE',
           'r_contN', 'r_corrN', 'g_contN', 'g_corrN', 'b_contN', 'b_corrN', 'nir_contN', 'nir_corrN']

# ===================================================
# read in data
all_pts = pd.read_csv(data_path)
itemids = list(all_pts[itemid_col].unique())  
crs = CRS.from_string(all_pts[crs_col][0]) # crs of dataframe
N = len(itemids)  # counter to finish

# ===================================================
sampled_pts = [] # sampled pts from each scene are collected here

# ===================================================
print('REMAINING: ', N, 'scenes', end="\r")
for i in range(len(itemids)):
    # ---------------------------------------
    # open raster reader for NAIP scene
    itemid = itemids[i]
    item = sr.get_item_from_id(itemid)    
    href = pc.sign(item.assets["image"].href)
    naip_rast_r = rioxr.open_rasterio(href) 

    pts_scene = all_pts.loc[all_pts['naip_id'] == itemid]
    
    if len(pts_scene) !=0:
        # create geodataframe with pts in scene
        pts_scene_df = sr.geodataframe_from_csv(df = pts_scene, 
                                                lon_label=lon_label, 
                                                lat_label=lat_label, 
                                                crs=crs)
        # convert pts to crs of NAIP scene        
        pts_col = pts_scene_df.to_crs(naip_rast_r.rio.crs).geometry
        
        # list of GLCM features for all pts in scene (elements are 1x(n_features) dataframes, one for each pt) 
        samples = []
        for pt in pts_col:
            
            # features for given point (list of 1x16 dataframes, each one represents features computed at a different window size)
            pt_samples = []
            for r in window_r:
                # make a small window out of raster, all bands
                row, col = rowcol(naip_rast_r.rio.transform(), pt.x, pt.y)
                windows = naip_rast_r[:, row-r:row+r+1, col-r:col+r+1]

                # calculate GLCM on that small window, all bands
                # calculate the contrast and correlation on that window (all angles)
                # add these as features for that pixel 
                
                pixel_feats = []
                for angle in angles:
                    for band in range(4):
                        glcm = graycomatrix(windows[band],
                                    distances=distances,
                                    angles=[angle])
                        pixel_feats.append(graycoprops(glcm, 'contrast')[0,0])
                        pixel_feats.append(graycoprops(glcm, 'correlation')[0,0])
                # this could be probably be done better wo making the dfs and just keeping track of the feature arrays
                d = pd.DataFrame(data=pixel_feats).T
                d.columns = [s + '_' +str(r*2+1) for s in columns ]
                pt_samples.append(d)
            samples.append(pd.concat(pt_samples, axis=1))
                       
        # ---------------------------------------
        # Add all derived spectral data to pts dataframe
        new_features = pd.concat(samples)
        pts = pd.concat([pts_scene, new_features.set_index(pts_col.index)], axis=1)  

        # -----------------------------
        # collect all points in the scene
        sampled_pts.append(pts)                    

    # ---------------------------------------
    # processing message
    N = N-1                
    print('REMAINING: ', N, 'scenes', end="\r")

clear_screen()
print('FINISHED PROCESSING')      

# ---------------------------------------
# create data frame with all points
sampled_pts = pd.concat(sampled_pts).sort_index()           

# ---------------------------------------
# save points
out_file_name = 'glcm_'+'-'.join([str(r) for r in window_r])+ '_'+ data_path.split('/')[-1]
sampled_pts.to_csv(os.path.join(out_dir, out_file_name),
                   index=False)

# ---------------------------------------
print(sampled_pts.columns)